In [39]:
from ipywidgets import *
from PIL import Image
import numpy as np
from skimage.draw import line
from IPython.display import display

In [40]:
class Tomograph:
    def __configure(self, img, *, alpha=1, n_detectors=1000, phi=np.pi):
        self.img = img
        self.__reshape_img()
        self.img = np.asarray(self.img)
        self.alpha = alpha
        self.n_detectors = n_detectors
        self.phi = phi
        self.n_steps = int(np.floor(180 / self.alpha))
        self.generated_sinogram = np.zeros((self.n_steps, self.n_detectors), dtype=int)
        self.bresenham_lines = []

    def __reshape_img(self):
        width, height = self.img.size
        new_size = max(width, height)
        self.img = self.img.resize((new_size, new_size))

    def __map_coords(self, angle):
        width = int(self.img.shape[0] / 2)
        x = int(np.cos(angle) * width) + width - 1
        y = -int(np.sin(angle) * width) + width - 1
        return x, y

    def get_sinogram(self):
        return Image.fromarray(self.generated_sinogram).convert('L')

    def __normalize_img(self, img):
        brightest_pixel = 0
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                if img[i][j] > brightest_pixel:
                    brightest_pixel = img[i][j]
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                img[i][j] = int(img[i][j] / brightest_pixel * 255)
        return img

    def __generate_sinogram(self):
        i = 0
        sl_filter = self.__hamming_filter(self.n_detectors)
        for angle in np.arange(0, np.pi, self.alpha * (np.pi / 180)):
            for detector in range(self.n_detectors):
                emitter_angle = angle + self.phi / 2 - detector * self.phi / (self.n_detectors - 1)
                emitter_coordX, emitter_coordY = self.__map_coords(emitter_angle)
                detector_angle = angle + np.pi - self.phi / 2 + detector * self.phi / (self.n_detectors - 1)
                detector_coordX, detector_coordY = self.__map_coords(detector_angle)

                rr, cc = line(emitter_coordX, emitter_coordY, detector_coordX, detector_coordY)
                self.generated_sinogram[i][detector] += self.img[rr, cc].sum()
                self.bresenham_lines.append([rr, cc])

            self.generated_sinogram[i] = np.array(np.convolve(self.generated_sinogram[i], sl_filter, 'same'))
            i += 1
            # print(i)

        self.generated_sinogram = self.__normalize_img(self.generated_sinogram)

    def reconstruct(self, n_images=1):
        reconstructed_img = np.zeros((self.img.shape[0], self.img.shape[1]))
        i = 0
        p = int(self.n_steps * self.n_detectors / n_images)
        generated_sinogram_flattened = self.generated_sinogram.flatten()

        for bresenham_line in self.bresenham_lines:
            rr, cc = bresenham_line
            reconstructed_img[rr, cc] += generated_sinogram_flattened[i]
            i += 1
            if i % p == 0:
                # print(i, p)
                norm_reconstructed_img = reconstructed_img.copy()
                norm_reconstructed_img = self.__normalize_img(norm_reconstructed_img)
                yield Image.fromarray(norm_reconstructed_img).convert('L')

    # Ram-Lak with hamming filter
    def __hamming_filter(self, length):
        filter = []
        ham = np.hamming(length)
        length = int(length/2)
        for i, j in zip(range(-length, length), ham):
            if i % 2 != 0:
                val = ((-4/np.pi**2)/(i**2)) * j
                filter.append(val)
            else:
                filter.append(0)
        filter[int(length)] = 1

        return filter

    def rmse(self, img1, img2):
        return np.sqrt(np.mean((img1-img2)**2))
    
    def run(self, source, *, alpha=1, n_detectors=1000, phi=np.pi, n_images=4):
        initial_img = Image.open(source).convert('L')
        self.__configure(initial_img, alpha=alpha, n_detectors=n_detectors, phi=phi)
        self.__generate_sinogram()
        path = 'Output/Sinogram/sinogram.jpg'
        self.get_sinogram().save(path)
        display(Image.open(path))
        i=0
        reconstructed_images = []
        for img in self.reconstruct(n_images=n_images):
            path = 'Output/Reconstructed_Image/part' + str(i) + '.jpg'
            img.save(path)
            img1 = open(path, 'rb').read()
            wi1 = widgets.Image(value=img1, format='png')
            reconstructed_images.append(wi1)
            i += 1
        display(widgets.HBox(reconstructed_images))


In [46]:
# config
SOURCE_IMAGE_PATH = 'Input/Kwadraty2.jpg'
NUMBER_OF_RECONSTRUCTED_IMAGES = 4
# config

tomograph = Tomograph()
interact(tomograph.run, source=fixed(SOURCE_IMAGE_PATH), n_images=fixed(NUMBER_OF_RECONSTRUCTED_IMAGES))


interactive(children=(IntSlider(value=1, description='alpha', max=3, min=-1), IntSlider(value=1000, descriptio…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>